In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
mpl.rcParams['font.family'] = 'serif'
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
indir_inter = os.path.join(rootdir, '_2_intermediate', 'data')
indir_prepro = os.path.join(rootdir, '_1_preprocessing', 'data', 'raw', 'gis')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [3]:
data = pd.read_csv(os.path.join(indir_inter, '_F_dist_religion_bch10.csv'))[['iso',
                                                         'bch10',
                                                         'major_religion',
                                                         'district',
                                                         'immg_18',
                                                         'n_immg_18']]
data = data[data.major_religion.isin(['Christian', 'Muslim', 'Traditional'])].reset_index(drop=True)
data = data[~((pd.isnull(data.immg_18)) & (pd.isnull(data.n_immg_18)))]

#####################################################################################

data['x'] = data.immg_18 * data.n_immg_18
del data['bch10']
data = data.groupby([data.iso, data.district, data.major_religion]).sum()
# del data['district']
data = data.reset_index(drop=False)
data['immg_18'] = data['x'] / data['n_immg_18']
del data['x'], data['n_immg_18']

data.index = [data.iso, data.district, data.major_religion]
del data['iso'], data['district'], data['major_religion']
data = data.unstack().reset_index(drop=False)
data.columns = [x[0] + x[1] for x in data.columns]

data = data[~((pd.isnull(data.immg_18Christian)) &
              (pd.isnull(data.immg_18Muslim)) &
              (pd.isnull(data.immg_18Traditional)))]

data.columns = ['iso', 'district', 'im18c', 'im18m', 'im18t']

In [4]:
ISO = sorted(list(set(data.iso)))

In [5]:
countries = gpd.read_file(os.path.join(indir_prepro, 'africa_countries', 'africa_countries.shp'))
afrregions = gpd.read_file(os.path.join(indir_prepro, 'districts_for_plotting', 'districts_for_plotting.shp'))[['iso', 'district', 'geometry']]
afrregions = afrregions[afrregions.iso.isin(ISO)].reset_index(drop=True).copy()
afrregions = pd.merge(afrregions, data, on=['iso', 'district'], how='left')
africa = afrregions[afrregions.geometry!=None].copy(deep=True)

In [6]:
countries['iso'] = countries.iso3
for c in africa.columns:
    if c not in countries.columns:
        countries[c] = np.nan
countries = countries[[c for c in africa.columns]]
countries = countries[~countries.iso.isin(list(set(africa.iso)) + ['ESP', 'PRT', 'FRA', 'YEM', 'SHN', 'ATF', 'REU'])]

In [7]:
africa = pd.concat([africa, countries], axis=0, ignore_index=True)

In [8]:
print('number of districts for which we observe upward IM for both Christians and Muslims aged 14-18     :', africa[(~pd.isnull(africa.im18c)) & (~pd.isnull(africa.im18m))].drop_duplicates(subset=['district']).shape[0])
print('number of districts for which we observe upward IM for both Christians and Traditionals aged 14-18:', africa[(~pd.isnull(africa.im18c)) & (~pd.isnull(africa.im18t))].drop_duplicates(subset=['district']).shape[0])

number of districts for which we observe upward IM for both Christians and Muslims aged 14-18     : 1861
number of districts for which we observe upward IM for both Christians and Traditionals aged 14-18: 1322


In [9]:
missing_kwds = dict(color='white', label='no Data')

titles = ['upward IM', 'upward IM', 'upward IM']
for idvar, variable in enumerate(['im18c', 'im18m', 'im18t']):
    title = titles[idvar]

    f, ax = plt.subplots(1,1, figsize=(10, 10))
    africa.plot(ax=ax, 
            column=variable,
            scheme='user_defined',
            classification_kwds={'bins':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]},
            cmap = 'YlOrRd',
            edgecolor='k',
            linewidth=0.1,
            legend=True, 
            missing_kwds=missing_kwds)

    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.15, 0.3, 0.2, 0.2))
    for txt in leg.texts:
        txti = re.sub('\.([0-9])0', r'.\1',  txt.get_text().replace(', ', ' - '))
        txt.set_text(txti)
    leg.set_title(title, prop={'size': 12, 'weight': 'bold'})
    
    f.savefig(outdir + f'/_4_heatmap_{variable}.pdf', bbox_inches='tight')
    plt.close(f)